In [5]:
import pandas as pd
import ast
import os
import csv
import numpy as np

def clean_and_split_data(input_file):
  print(f"--- BẮT ĐẦU XỬ LÝ FILE: {input_file} ---")
  
  # Đọc file dữ liệu gốc
  try:
    df = pd.read_csv(input_file)
    print(f"Đã đọc file thành công. Tổng số dòng: {len(df)}")
  except FileNotFoundError:
    print("Lỗi: Không tìm thấy file đầu vào!")
    return

  # Folder xuất dữ liệu (để khớp với docker-compose của bạn)
  output_dir = '../data/ingestion'
  os.makedirs(output_dir, exist_ok=True)

  # --- 1. XỬ LÝ HIGHLIGHTS & INGREDIENTS (Về dạng TEXT thuần) ---
  def process_highlights(val):
    if pd.isna(val) or val == '': return None
    try:
      if isinstance(val, str) and val.startswith('['):
        clean_list = ast.literal_eval(val)
        if isinstance(clean_list, list): return ", ".join(clean_list)
      return str(val)
    except: return None

  def process_ingredients(val):
    if pd.isna(val) or val == '': return None
    try:
      if isinstance(val, str) and val.startswith('['):
        clean_list = ast.literal_eval(val)
        if isinstance(clean_list, list) and len(clean_list) > 0:
          item = clean_list[0]
          if isinstance(item, dict): return item.get('ingredients')
      return str(val)
    except: return None

  print("-> Đang xử lý cột highlights và ingredients...")
  df['highlights'] = df['highlights'].apply(process_highlights)
  df['ingredients'] = df['ingredients'].apply(process_ingredients)

  # --- 2. TẠO BRANDS.CSV ---
  print("-> Đang tách bảng BRANDS...")
  unique_brands = df['brand_name'].dropna().unique()
  brands_df = pd.DataFrame(unique_brands, columns=['brand_name'])
  brands_df.insert(0, 'brand_id', range(1, 1 + len(brands_df)))
  brands_df.to_csv(f'{output_dir}/brands.csv', index=False)
  brand_map = brands_df.set_index('brand_name')['brand_id'].to_dict()

  # --- 3. TẠO PRODUCTS.CSV ---
  print("-> Đang tách bảng PRODUCTS...")
  df['brand_id'] = df['brand_name'].map(brand_map)
  cols_to_keep = ['brand_id', 'product_name', 'description', 'ingredients', 'highlights']
  # GroupBy product_id, lấy dòng đầu tiên
  products_df = df.groupby('product_id')[cols_to_keep].first().reset_index()
  products_df.to_csv(f'{output_dir}/products.csv', index=False)

  # --- 4. TẠO VARIANTS.CSV (XỬ LÝ MÀU SẮC THEO YÊU CẦU CỦA BẠN) ---
  print("-> Đang tách bảng VARIANTS và làm sạch Color...")
  variant_cols = ['uniq_id', 'product_id', 'color', 'size', 'price', 'availability', 'primary_image_url']
  variants_df = df[variant_cols].copy()

  def clean_color_custom(val):
    if pd.isna(val): return None
    val_str = str(val).strip()
    
    # Case 1: Đang bị bao bởi ngoặc kép (VD: "Màu A, Màu B")
    if val_str.startswith('"') and val_str.endswith('"'):
      # Bỏ ngoặc kép đầu đuôi
      content = val_str[1:-1]
      # Thay dấu phẩy bằng gạch ngang
      return content.replace(', ', ' - ').replace(',', '-')
    
    # Case 2: Không có ngoặc nhưng có dấu phẩy (VD: Màu A, Màu B) -> Cũng thay luôn
    if ',' in val_str:
      return val_str.replace(', ', ' - ').replace(',', '-')
          
    # Case 3: Bình thường (VD: Red) -> Giữ nguyên
    return val_str

  variants_df['color'] = variants_df['color'].apply(clean_color_custom)
  
  # LƯU FILE: Dùng QUOTE_MINIMAL (Mặc định).
  # Vì ta đã thay hết dấu phẩy trong color thành dấu gạch ngang rồi, 
  # nên pandas sẽ thấy cột color là text "sạch" và tự động KHÔNG thêm dấu ngoặc kép nữa.
  variants_df.to_csv(f'{output_dir}/variants.csv', index=False, quoting=csv.QUOTE_MINIMAL)

  print(f"\n--- XỬ LÝ HOÀN TẤT! ---")
  print(f"File variants.csv đã được làm sạch theo logic: ',' -> '-'")

In [6]:
clean_and_split_data('../data/processed/processed_lipstick_data.csv')

--- BẮT ĐẦU XỬ LÝ FILE: ../data/processed/processed_lipstick_data.csv ---
Đã đọc file thành công. Tổng số dòng: 2081
-> Đang xử lý cột highlights và ingredients...
-> Đang tách bảng BRANDS...
-> Đang tách bảng PRODUCTS...
-> Đang tách bảng VARIANTS và làm sạch Color...

--- XỬ LÝ HOÀN TẤT! ---
File variants.csv đã được làm sạch theo logic: ',' -> '-'
